In [6]:
# Импорт всех необходимых библиотек
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from wordcloud import WordCloud
import numpy as np

# Загрузка данных
data = pd.read_csv('cleaned_comments.csv', sep='\t', encoding='utf-8')

# Подготовка данных
data['positive'] = data[['Нравится качество выполнения заявки', 
                        'Нравится качество работы сотрудников',
                        'Нравится скорость отработки заявки',
                        'Понравилось выполнение заявки']].any(axis=1).astype(int)

comments = data.iloc[:, 2:6].fillna('').apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
target = data['positive']

# Визуализация распределения классов
plt.figure(figsize=(8, 6))
sns.countplot(x=target)
plt.title('Распределение положительных и отрицательных отзывов')
plt.xlabel('Класс (0 - отрицательный, 1 - положительный)')
plt.ylabel('Количество')
plt.show()

# Облака слов
positive_text = ' '.join(comments[target == 1])
negative_text = ' '.join(comments[target == 0])

wordcloud_positive = WordCloud(width=800, height=400, background_color='white').generate(positive_text)
wordcloud_negative = WordCloud(width=800, height=400, background_color='white').generate(negative_text)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
ax1.imshow(wordcloud_positive, interpolation='bilinear')
ax1.set_title('Положительные отзывы')
ax1.axis('off')
ax2.imshow(wordcloud_negative, interpolation='bilinear')
ax2.set_title('Отрицательные отзывы')
ax2.axis('off')
plt.show()

# Разделение данных и векторизация
X_train, X_test, y_train, y_test = train_test_split(comments, target, test_size=0.2, random_state=42)

vectorizer = CountVectorizer(stop_words=['спасибо', 'большое', 'огромное'], max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение модели
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

# Оценка модели
y_pred = nb_classifier.predict(X_test_vec)
print(classification_report(y_test, y_pred))

# Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=['Отрицательные', 'Положительные'])
disp.plot(cmap='Blues')
plt.title('Матрица ошибок классификатора')
plt.show()

# ROC-кривая
y_prob = nb_classifier.predict_proba(X_test_vec)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label='ROC кривая (площадь = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Ложноположительная частота')
plt.ylabel('Истинноположительная частота')
plt.title('ROC-кривая')
plt.legend(loc="lower right")
plt.show()

# Важность признаков
features_df = pd.DataFrame({
    'word': vectorizer.get_feature_names_out(),
    'positive_coef': nb_classifier.feature_log_prob_[1],
    'negative_coef': nb_classifier.feature_log_prob_[0]
})

top_positive = features_df.sort_values('positive_coef', ascending=False).head(20)
top_negative = features_df.sort_values('negative_coef', ascending=False).head(20)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 10))
sns.barplot(x='positive_coef', y='word', data=top_positive, palette='viridis', ax=ax1)
ax1.set_title('Топ-20 слов для положительных отзывов')
ax1.set_xlabel('Логарифмическая вероятность')

sns.barplot(x='negative_coef', y='word', data=top_negative, palette='magma', ax=ax2)
ax2.set_title('Топ-20 слов для отрицательных отзывов')
ax2.set_xlabel('Логарифмическая вероятность')
plt.tight_layout()
plt.show()

# Длина комментариев
data['comment_length'] = comments.apply(len)
plt.figure(figsize=(10, 6))
sns.boxplot(x='positive', y='comment_length', data=data)
plt.title('Распределение длины комментариев по классам')
plt.xlabel('Класс (0 - отрицательный, 1 - положительный)')
plt.ylabel('Длина комментария')
plt.show()

# Кривая обучения
plot_learning_curve(nb_classifier, "Кривая обучения (Наивный Байес)", 
                    X_train_vec, y_train)
plt.show()

KeyError: "['Нравится скорость отработки заявки'] not in index"